STEP 1:
First we will have to create a table template for our user database. Therefore, we initialize the table first. We have also assigned the size to each cell based on the maximum length of each cell in our excel sheet. 

In [ ]:
#Initialize the Table
import pymysql
mydb = pymysql.connect( host = 'localhost' ,  user ="root" ,  passwd = "sankalp1911" , db = "project_db")  
cursor = mydb.cursor() 

q = """CREATE TABLE profiles(
   Name varchar(32) NOT NULL,
   Intro varchar(255) NOT NULL,
    Location varchar(32) NOT NULL,
    Job varchar(255) NOT NULL,
    About varchar(5000) NOT NULL,
    Education varchar(5000) NOT NULL,
    Skills varchar(5000) NOT NULL,
    URL varchar(255) NOT NULL
)"""

cursor.execute(q)
    
cursor.close()
mydb.commit()
mydb.close()

STEP 2:
Now we have to insert the data from our excel/csv file to our database table. 

In [15]:
#Insert data into the table

import xlrd
import pymysql

xl_data = xlrd.open_workbook('C:/Users/Sankalp/Desktop/Problem_Sample Linkedin Data.xlsx')
sheet = xl_data.sheet_by_index(0)
mydb = pymysql.connect( host = 'localhost' ,  user ="root" ,  passwd = "sankalp1911" , db = "project_db")  
cursor = mydb.cursor() 


query = """INSERT INTO profiles (Name, Intro, Location, Job, About, Education, Skills, URL) 
                           VALUES 
                           (%s, %s, %s, %s, %s, %s, %s, %s) """
#n1=n2=n3=n4=n5=n6=n7=n8=0
for i in range(1,sheet.nrows):
    Name = sheet.cell(i,0).value
    Intro = sheet.cell(i,1).value
    Location = sheet.cell(i,2).value
    Job = sheet.cell(i,3).value
    About = sheet.cell(i,4).value
    Education = sheet.cell(i,5).value
    Skills = sheet.cell(i,6).value
    URL = sheet.cell(i,7).value
#     n1 = max(n1,len(Name))
#     n2 = max(n2,len(Intro))
#     n3 = max(n3,len(Location))
#     n4 = max(n4,len(Job))
#     n5 = max(n5,len(About))
#     n6 = max(n6,len(Education))
#     n7 = max(n7,len(Skills))
#     n8 = max(n8,len(URL))
    
    values = (Name,Intro,Location,Job,About,Education,Skills,URL)
    
    cursor.execute(query,values)
#print(n1,n2,n3,n4,n5,n6,n7,n8)    
cursor.close()
mydb.commit()
mydb.close()

STEP 3:
Now we query the database based on the input string 't'. 

In [3]:
import pymysql
from tabulate import tabulate
mydb = pymysql.connect( host = 'localhost' ,  user ="root" ,  passwd = "sankalp1911" , db = "project_db")  

cursor = mydb.cursor() 

t = str(input("Enter a Text Query:"))
f_at = t.find(' at ')
f_in = t.find(' in ')

if((f_at == -1) & (f_in==-1)):
    q = """SELECT * FROM profiles
WHERE Location LIKE '%"""+t+"""%'
   OR Job LIKE '%"""+t+"""%'
   OR Skills LIKE '%"""+t+"""%'"""
    cursor.execute(q)
    res = cursor.fetchall()
    if(len(res) == 0):
        cursor.close()
        quer = """ALTER TABLE profiles ADD FULLTEXT(Location, Job, Skills)"""
        query = """SELECT * FROM profiles WHERE MATCH(Location, Job, Skills)
                AGAINST("""+"\'"+t+"\'"+""" IN BOOLEAN MODE)"""
        cursor = mydb.cursor() 
        cursor.execute(quer)
        cursor.execute(query)
        res = cursor.fetchall()
        
        

    
elif((f_at != -1) & (f_in==-1)):
    before_at = t[0:f_at]
    after_at = t[f_at+4:]
    #print(before_at,after_at)
    q = """SELECT * FROM profiles
WHERE ((Job LIKE '%"""+t+"""%')
   OR ((Job LIKE '%"""+before_at+"""%'
   OR Skills LIKE '%"""+before_at+"""%')
   AND Location LIKE '%"""+after_at+"""%'))"""
    cursor.execute(q)
    res = cursor.fetchall()
    
    
elif((f_at == -1) & (f_in!=-1)):
    before_in = t[0:f_in]
    after_in = t[f_in+4:]
    #print(before_in,after_in)
    q = """SELECT * FROM profiles
WHERE ((Location LIKE '%"""+after_in+"""%')
   AND ((Job LIKE '%"""+before_in+"""%'
   OR Skills LIKE '%"""+before_in+"""%')))"""
    cursor.execute(q)
    res = cursor.fetchall()
    
    
else:
    between_at_in = t[f_at+4:f_in]
    after_in = t[f_in+4:]
    before_at = t[0:f_at]
    before_in = t[0:f_in]
    q = """SELECT * FROM profiles
WHERE (Location LIKE '%"""+after_in+"""%'
   AND ((Job LIKE '%"""+before_in+"""%'
   OR Skills LIKE '%"""+before_at+"""%')))"""
    cursor.execute(q)
    res = cursor.fetchall()



# print(l,j)
# quer = """ALTER TABLE profiles ADD FULLTEXT(Location, Job, Skills)"""
# query = """SELECT * FROM profiles WHERE MATCH(Location, Job, Skills)
# AGAINST("""+"\'"+t+"\'"+""" IN BOOLEAN MODE)"""

# cursor.execute(quer)
#cursor.execute(q)

#res = cursor.fetchall()
# print(res[0])
print(str(len(res))+" relevant records found!\n")
res = list(res)
# k=0

for i in range(len(res)):
    res[i] = list(res[i])
    del res[i][4 : 7]
    
    
for r in res:
    #k=k+1
    #print("Record:"+str(k))
#     print(r)
#     print('\n')
    r[1] = r[1][0:10] + "[...]"
    r[2] = r[2][0:15] + "[...]"
    r[3] = r[3][8:19] + "[...]"
   
    
    
tres = tabulate(res, headers=['Name', 'Intro', 'Location', 'Job', 'URL'], tablefmt='psql')
print(tres)
cursor.close()
mydb.commit()
mydb.close()

Enter a Text Query:San Francisco
6 relevant records found!

+-------------------+-----------------+----------------------+-----------------+----------------------------------------------+
| Name              | Intro           | Location             | Job             | URL                                          |
|-------------------+-----------------+----------------------+-----------------+----------------------------------------------|
| Gopal PG          | Investor &[...] | San Francisco B[...] | Angel & Ve[...] | https://www.linkedin.com/in/pg1976/          |
| Yashwanth Hemaraj | Partner, I[...] | San Francisco B[...] | Partner at[...] | https://www.linkedin.com/in/yhemaraj/        |
| Gopal PG          | Investor &[...] | San Francisco B[...] | Angel & Ve[...] | https://www.linkedin.com/in/pg1976/          |
| Yashwanth Hemaraj | Partner, I[...] | San Francisco B[...] | Partner at[...] | https://www.linkedin.com/in/yhemaraj/        |
| Shaun Mehra       | Healthcare[...] | San 

STEP 4: We will develop frontend GUI for the above code

In [4]:
import tkinter as tk

from tabulate import tabulate
root= tk.Tk()

canvas1 = tk.Canvas(root, width = 400, height = 300)
canvas1.pack()

entry1 = tk.Entry (root) 
canvas1.create_window(200, 140, window=entry1)

def search ():  
    t = entry1.get()
    mydb = pymysql.connect( host = 'localhost' ,  user ="root" ,  passwd = "sankalp1911" , db = "project_db")  

    cursor = mydb.cursor() 

    #t = str(input("Enter a Text Query:"))
    f_at = t.find(' at ')
    f_in = t.find(' in ')

    if((f_at == -1) & (f_in==-1)):
        q = """SELECT * FROM profiles
    WHERE Location LIKE '%"""+t+"""%'
       OR Job LIKE '%"""+t+"""%'
       OR Skills LIKE '%"""+t+"""%'"""
        cursor.execute(q)
        res = cursor.fetchall()
        if(len(res) == 0):
            cursor.close()
            quer = """ALTER TABLE profiles ADD FULLTEXT(Location, Job, Skills)"""
            query = """SELECT * FROM profiles WHERE MATCH(Location, Job, Skills)
                    AGAINST("""+"\'"+t+"\'"+""" IN BOOLEAN MODE)"""
            cursor = mydb.cursor() 
            cursor.execute(quer)
            cursor.execute(query)
            res = cursor.fetchall()




    elif((f_at != -1) & (f_in==-1)):
        before_at = t[0:f_at]
        after_at = t[f_at+4:]
        #print(before_at,after_at)
        q = """SELECT * FROM profiles
    WHERE ((Job LIKE '%"""+t+"""%')
       OR ((Job LIKE '%"""+before_at+"""%'
       OR Skills LIKE '%"""+before_at+"""%')
       AND Location LIKE '%"""+after_at+"""%'))"""
        cursor.execute(q)
        res = cursor.fetchall()


    elif((f_at == -1) & (f_in!=-1)):
        before_in = t[0:f_in]
        after_in = t[f_in+4:]
        #print(before_in,after_in)
        q = """SELECT * FROM profiles
    WHERE ((Location LIKE '%"""+after_in+"""%')
       AND ((Job LIKE '%"""+before_in+"""%'
       OR Skills LIKE '%"""+before_in+"""%')))"""
        cursor.execute(q)
        res = cursor.fetchall()


    else:
        between_at_in = t[f_at+4:f_in]
        after_in = t[f_in+4:]
        before_at = t[0:f_at]
        before_in = t[0:f_in]
        q = """SELECT * FROM profiles
    WHERE (Location LIKE '%"""+after_in+"""%'
       AND ((Job LIKE '%"""+before_in+"""%'
       OR Skills LIKE '%"""+before_at+"""%')))"""
        cursor.execute(q)
        res = cursor.fetchall()


    print(str(len(res))+" relevant records found!\n")
    res = list(res)
    # k=0

    for i in range(len(res)):
        res[i] = list(res[i])
        del res[i][4 : 7]


    for r in res:
        #k=k+1
        #print("Record:"+str(k))
    #     print(r)
    #     print('\n')
        r[1] = r[1][0:10] + "[...]"
        r[2] = r[2][0:15] + "[...]"
        r[3] = r[3][8:19] + "[...]"

    res.insert(0, ['Name','Intro','Location', 'Job','URL'])
    cursor.close()
    mydb.commit()
    mydb.close()
    
    #label1 = tk.Label(root, text= tres)
    class TabulateLabel(tk.Label):
        def __init__(self, parent, data, **kwargs):
            super().__init__(parent, 
                             font=('Consolas', 10), 
                             justify=tk.LEFT, anchor='nw', **kwargs)

            text = tabulate(data, headers='firstrow', tablefmt='psql', showindex=False)
            self.configure(text=text)


    class App(tk.Tk):
        def __init__(self):
            super().__init__()
            TabulateLabel(self, data=res, bg='white').grid(sticky='ew')

    if __name__ == "__main__":
        App().mainloop()

    
    #canvas1.create_window(200, 230, window=label1)
    
button1 = tk.Button(text='Search', command=search)
canvas1.create_window(200, 180, window=button1)

root.mainloop()

5 relevant records found!

